curl -fsSL https://ollama.com/install.sh | sh

ollama run gemma2:2b

!pip install ollama

!pip install -U langchain-ollama

pip install langchain-huggingface ~5 min

!pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j

!pip install rank_bm25

In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY'] = 

In [2]:
import ollama

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cuda:0'}
encode_kwargs = {'normalize_embeddings': False}
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
    
)

/home/valeriy/python_projects/RAG/RAG_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
file_path = 'data/txts/cleaned.txt'
with open(file_path, 'r', encoding="utf-8") as f:
    lines = f.readlines()

In [5]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="belfoot_collection",
    embedding_function=hf_embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [6]:
from uuid import uuid4

from langchain_core.documents import Document
    

documents = [Document (page_content=text, id=idx) for idx, text in enumerate(lines)]
uuids = [str(uuid4()) for _ in range(len(documents))]

#vector_store.add_documents(documents=documents, ids=uuids) #CPU - >1.5 hours, GPU - 15min

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
template = """Question: {question}

Context: {context}

Answer:"""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model="gemma2:2b")


In [16]:

from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

In [17]:
TOP_K_SPARSE = 7
TOP_K_DENSE = 7
NUMBER_OF_ATTEMPTS = 3

In [18]:
from langchain_community.retrievers import BM25Retriever

bm_25_retriever = BM25Retriever.from_documents(documents, k=TOP_K_SPARSE)


In [19]:
# Define state for application
class State(TypedDict):
    system: str
    question: str
    context: List[Document]
    answer: str
    attempts: int

In [20]:
from typing import List, Dict, Literal
def should_continue(state: State) -> Literal["YES", "NO"]:
    """Complex routing logic"""

    if state['attempts'] >= NUMBER_OF_ATTEMPTS:
        return "end"
    
    answer_stripped = state['answer'].strip()
    if answer_stripped == "YES":
        return "end"
    elif answer_stripped == "NO":
        return "continue"
    else:
        print(answer_stripped)
        assert False

def redirect_reject(state: State) -> Literal["retrival", "generation"]:
    """Complex routing logic"""

    answer_stripped = state['answer'].strip()
    if answer_stripped == "RETRIVAL":
        return "retrival"
    elif answer_stripped == "GENERATION":
        return "generation"
    else:
        print(answer_stripped)
        assert False

In [ ]:
from typing import List, TypedDict
from langchain.schema import Document
from langgraph.graph import StateGraph, START, END

def rewrite_query(state: State):
    system = """You are an expert at converting user questions into database queries. \
    You have access to a database of telegram posts about Belarusian football for building LLM-powered applications. \

    Perform query decomposition. Given a user question, break it down into distinct sub questions that \
    you need to answer in order to answer the original question.

    If there are acronyms or words you are not familiar with, do not try to rephrase them.
    
    Do not add any additional information or explanation"""
    
    messages = {'system': system,
                'question': state['question'],
                'answer':''}

    resp = model.invoke(str(messages))

    return {"question":resp}

def retrieve(state: State):
    bm25_docs = bm_25_retriever.invoke(state["question"])
    vector_docs = vector_store.similarity_search(state["question"], k=TOP_K_DENSE)
    
    # Merge and remove duplicates
    doc_dict = {doc.page_content: doc for doc in bm25_docs + vector_docs}
    combined_docs = list(doc_dict.values())

    return {"context": combined_docs, "attempts": 0}

def generate(state: State):
    
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = model.invoke(messages)
    
    return {"answer": response, 'attempts':state['attempts']+1}

def verify_answer(state: State):
    verification_prompt = {
        "question": state["question"],
        "context": "\n\n".join(doc.page_content for doc in state["context"]),
        "answer": state["answer"]
    }
    verification_result = model.invoke(f"How do you think, does your answer answers the question correctly and completely? Double-check alignment with context in order not to mislead. Respond with 'YES' or 'NO'. \
                                       \n\n{verification_prompt} \
                                       \n\nSingle word: YES or NO")

    return {"answer":verification_result}

def fault_reason(state: State):
    verification_prompt = {
        "question": state["question"],
        "context": "\n\n".join(doc.page_content for doc in state["context"]),
        "answer": state["answer"]
    }
    verification_result = model.invoke(f"Why do you think answer is not good enough?.\n\n{verification_prompt} \
                                       \n\nRespond with single word : RETRIVAL if you think that retrived data is not enough; and GENERATION if retrived data is good enough and problem is in generation stage")

    return {"answer":verification_result}

# Build the LangGraph workflow
graph_builder = StateGraph(State)

graph_builder.add_node("retrieve", retrieve)
graph_builder.add_node("generate", generate)
graph_builder.add_node("verify_answer", verify_answer)
graph_builder.add_node("fault_reason", fault_reason)
graph_builder.add_node("rewrite_query", rewrite_query)

# Define edges (flow control)
graph_builder.add_edge(START, "retrieve")
graph_builder.add_edge("retrieve", "generate")
graph_builder.add_edge("generate", "verify_answer")
graph_builder.add_edge("rewrite_query", "retrieve")


graph_builder.add_conditional_edges(
    "verify_answer",
    should_continue,
    {
        "end": END,
        "continue": "fault_reason"
    }
)

graph_builder.add_conditional_edges(
    "fault_reason",
    redirect_reject,
    {
        "generation": 'generate',
        "retrival": "rewrite_query"
    }
)


# Compile the graph
graph = graph_builder.compile()


In [30]:
response = graph.invoke({"question": "Best player of Belarus in 2025: "})
print(response["answer"])

YES 



In [25]:
model.invoke("Please, list all Neman players of all time")

'I cannot provide you with a complete list of every player in the history of the Neman club. \n\nHere\'s why:\n\n* **Data availability:** Comprehensive historical data for all basketball clubs is not readily available online. This includes roster information, especially for smaller or amateur teams like Neman.  \n* **Privacy and data protection:**  Sharing personal details of past players without their consent would be a breach of privacy. \n\n\n**Where you might find some information:**\n\n* **Neman website:** Check the club\'s official website (if they have one) for historical information, perhaps under sections like "History," "About us," or "Club Archives."\n* **Basketball-related databases:** Some online basketball databases exist (like Eurobasket.com or FIBA.basketball), which may list some teams and their history, but it\'s unlikely to include every single player of all time at a club like Neman. \n\nI hope this helps! \n'